# CSC4008-tut9
## K-Means & PCA

##Review of K-Means##

KMeans is an unsupervised machine learning algorithm that is commonly used for data clustering. The goal of the algorithm is to partition a dataset into a specified number of clusters $k$. The process can be divided into five steps.

* Initialize: Choose the number of clusters $k$ and randomly initialize $k$ cluster centroids in the feature space.

* Assign Data Points to Clusters: For each data point, calculate the distance between the data point and each of the $k$ centroids. Assign the data point to the cluster associated with the nearest centroid.

* Update Centroids: After all data points have been assigned to clusters, calculate the mean of all data points within each cluster to obtain a new centroid for each cluster.

* Repeat: Repeat the above two steps until convergence, which is typically determined by a maximum number of iterations or when the centroids no longer move significantly.

* Output: The final centroids represent the k clusters, and the data points are classified into their corresponding clusters.

**The initial placement of centroids** and **the number of cluster k** have large effects on the result of the algorithm. We can use **silhouette score** to measure the result. 

##Mathematical insight of K-Means##

The process of K-Means is actually an optimization. Given the data set $\{x_i\}_{i=1}^n$, K-means aims to find cluster $c = \{c_j\}_{j=1}^k$ and assignment $r$, by minimizing the within-cluster variance, which is the sum of squared distances of data points to their assigned cluster centers.
$$\mathop{\min}_{c,r} \sum_{i}^{n} \sum_{j}^{k} r_{ij}(x_i-c_j)^2$$
$$s.t. \quad r \in \{0,1\}^{n \times k}, \quad \sum_{j}^{k} r_{ij}=1$$
Here $r_{ij}=1$ denotes $x_i$ is assigned to cluster $j$  
To optimize such a problem, the major idea is to update $c$ and $r$ alternatively:
* Given the cluster centers $c$, update the assignments $r$
* Given the assignments $r$, update the cluster centers $c$  

These two steps corresponding to the second and third step of K-Means.
* Assignments: Given the cluster centers $c$, update the assignments $r$ by
solving the following sub-problem
$$\mathop{\min}_{r} \sum_{i}^{n} \sum_{j}^{k} r_{ij}(x_i-c_j)^2$$
$$s.t. \quad r \in \{0,1\}^{n \times k}, \quad \sum_{j}^{k} r_{ij}=1$$  
Note that the assignment for each data $x_i$ can be solved independently. So we can further split the sub-problem into $n$ optimization problems.  
$$\mathop{\min}_{r_i} \sum_{j}^{k} r_{ij}(x_i-c_j)^2$$
$$s.t. \quad r_i = [0,...,1,...,0]$$
It is easy to know that assign $x_i$ to the closest cluster is the optimal solution.  
* Updating centroids: Given the assignments r, update the cluster centers c:
$$\mathop{\min}_{c} \sum_{i}^{n} \sum_{j}^{k} r_{ij}(x_i-c_j)^2$$
Note that $c_j$ can be optimized independently. So we can further split the sub-problem into $k$ optimization problems.  
$$\mathop{\min}_{c_j} \sum_{i}^{n} r_{ij}(x_i-c_j)^2$$
The problem can be characterized by finding the cluster center given the coordinates of points in that cluster. So we can just set the derivative of the objective function to 0 to find the optimal solution.
$$\sum_{i}^{n} -2r_{ij}(x_i-c_j) = 0$$
reformulate the equation, we have:
$$c_j = \frac{\sum_{i}^{n} r_{ij}x_i}{\sum_{i}^{n} r_{ij}}$$
The resulting $c_j$ is definitely the mean of points in that cluster.

##Silhouette score##

Given a clustering, for a single sample i, we define:
* a(i): The mean distance between the sample and all other points in the same
cluster.
* b(i): The mean distance between the sample and all other points in the next
nearest cluster.  

Silhouette coefficient $s$ for sample i is formulated as:
$$s(i) = \frac{b(i) - a(i)}{max\{a(i),b(i)\}}$$
Silhouette coefficient $s$ for a set of samples is given as the mean of the
Silhouette coefficient for each sample.

##PCA##

PCA is a dimensionality reduction technique commonly used in ML to transform high-dimensional data into a lower-dimensional representation while retaining the information of original data as much as possible. 

* Data Preparation: Prepare the data by organizing it into a matrix where rows represent data points and columns represent features.

* Data Standardization: Standardize the data by subtracting the mean and dividing by the standard deviation for each feature.

* Covariance Matrix Calculation: Compute the covariance matrix of the standardized data, which is a symmetric matrix that represents the pairwise covariance between all pairs of features.

* Eigendecomposition: Perform eigendecomposition on the covariance matrix to obtain its eigenvalues and eigenvectors. 

* Principal Component Selection: Sort the eigenvalues in descending order and select the top $k$ eigenvectors (principal components) that desired.

* Projection: Project the original data onto the selected principal components by taking the dot product between the standardized data and the selected eigenvectors. This results in a new lower-dimensional representation of the data, where the number of dimensions is reduced from the original number of features to $k$.

### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 15.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=fa943fd98eeba875894535912459d906232a67e78e515709d9b5373ecfb70c62
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenh

Now we import some of the libraries usually needed by our workload.





In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [ ]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

### Data Preprocessing

In this Colab, rather than downloading a file from Google Drive, we will load a famous machine learning dataset, the [Breast Cancer Wisconsin dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html), using the ```scikit-learn``` datasets loader.

In [ ]:
from sklearn.datasets import load_breast_cancer
breast_cancer = load_breast_cancer()

For convenience, given that the dataset is small, we first 

*   construct a Pandas dataframe
*   tune the schema
*   and convert it into a Spark dataframe.

In [ ]:
from pyspark.ml.linalg import Vectors
pd_df = pd.DataFrame(breast_cancer.data, columns=breast_cancer.feature_names)
df = spark.createDataFrame(pd_df)

def set_df_columns_nullable(spark, df, column_list, nullable=False):
  for struct_field in df.schema:
    if struct_field.name in column_list:
      struct_field.nullable = nullable
  df_mod = spark.createDataFrame(df.rdd, df.schema)
  return df_mod

df = set_df_columns_nullable(spark, df, df.columns)
df = df.withColumn('features', array(df.columns))
vectors = df.rdd.map(lambda row: Vectors.dense(row.features))

df.printSchema()

root
 |-- mean radius: double (nullable = false)
 |-- mean texture: double (nullable = false)
 |-- mean perimeter: double (nullable = false)
 |-- mean area: double (nullable = false)
 |-- mean smoothness: double (nullable = false)
 |-- mean compactness: double (nullable = false)
 |-- mean concavity: double (nullable = false)
 |-- mean concave points: double (nullable = false)
 |-- mean symmetry: double (nullable = false)
 |-- mean fractal dimension: double (nullable = false)
 |-- radius error: double (nullable = false)
 |-- texture error: double (nullable = false)
 |-- perimeter error: double (nullable = false)
 |-- area error: double (nullable = false)
 |-- smoothness error: double (nullable = false)
 |-- compactness error: double (nullable = false)
 |-- concavity error: double (nullable = false)
 |-- concave points error: double (nullable = false)
 |-- symmetry error: double (nullable = false)
 |-- fractal dimension error: double (nullable = false)
 |-- worst radius: double (nullable

With the next cell, we build the two data structures that we will be using throughout this Colab:


*   ```features```, a dataframe of Dense vectors, containing all the original features in the dataset;
*   ```labels```, a series of binary labels indicating if the corresponding set of features belongs to a subject with breast cancer, or not.



In [ ]:
features = spark.createDataFrame(vectors.map(Row), ["features"])
labels = np.array(breast_cancer.target)

### Your task

You are now ready to cluster the data with the [K-means](https://spark.apache.org/docs/latest/ml-clustering.html) algorithm included in MLlib (Spark's Machine Learning library).
Set the ```k``` parameter to **2**, fit the model, and the compute the [Silhouette score](https://en.wikipedia.org/wiki/Silhouette_(clustering)) (i.e., a measure of quality of the obtained clustering).  

**IMPORTANT:** use the MLlib implementation of the Silhouette score (via ```ClusteringEvaluator```).

In [ ]:
# YOUR CODE HERE
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
def k_means(feature):
  #train a k-means model
  kmeans = KMeans().setK(2).setSeed(1)
  model = kmeans.fit(feature)
  #make predictions
  predictions = model.transform(feature)
  #evaluate clustering by computing Silhouette score
  evaluator = ClusteringEvaluator()
  silhouette = evaluator.evaluate(predictions)
  predictions.show(10)
  print("Silhouette with squared euclidean distance =", str(silhouette))
  return predictions

predictions = k_means(features)


+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|[17.99,10.38,122....|         1|
|[20.57,17.77,132....|         1|
|[19.69,21.25,130....|         1|
|[11.42,20.38,77.5...|         0|
|[20.29,14.34,135....|         1|
|[12.45,15.7,82.57...|         0|
|[18.25,19.98,119....|         1|
|[13.71,20.83,90.2...|         0|
|[13.0,21.82,87.5,...|         0|
|[12.46,24.04,83.9...|         0|
+--------------------+----------+
only showing top 10 rows

Silhouette with squared euclidean distance = 0.8342904262826145


Take the predictions produced by K-means, and compare them with the ```labels``` variable (i.e., the ground truth from our dataset).  

Compute how many data points in the dataset have been clustered correctly (i.e., positive cases in one cluster, negative cases in the other).

**HINT**: you can use ```np.count_nonzero(series_a == series_b)``` to quickly compute the element-wise comparison of two arrays.

**IMPORTANT**: K-means is a clustering algorithm, so it will not output a label for each data point, but just a cluster identifier!  As such, label ```0``` does not necessarily match the cluster identifier ```0```.


In [ ]:
# YOUR CODE HERE
def calculate_TP(prediction,label,pca=False):
  predict_lst = prediction.toPandas()['prediction'].values
  n_correct_1 = np.count_nonzero(predict_lst == label)
  n_correct_2 = np.count_nonzero(1-predict_lst == label)
  n_correct = np.max([n_correct_1,n_correct_2])
  if pca:
    print("Number of data points that have been clustered correctly after PCA:",n_correct)
    print("Percent of data points that have been clustered correctly after PCA:",n_correct/len(label))
  else:
    print("Number of data points that have been clustered correctly:",n_correct)
    print("Percent of data points that have been clustered correctly:",n_correct/len(label))

calculate_TP(predictions,labels)
    

Number of data points that have been clustered correctly: 486
Percent of data points that have been clustered correctly: 0.8541300527240774


Now perform dimensionality reduction on the ```features``` using the [PCA](https://spark.apache.org/docs/latest/ml-features.html#pca) statistical procedure, available as well in MLlib.

Set the ```k``` parameter to **2**, effectively reducing the dataset size of a **15X** factor.

In [ ]:
# YOUR CODE HERE
from pyspark.ml.feature import PCA
pca = PCA(k=2,inputCol='features')
pca_model = pca.fit(features)
pca_model.setOutputCol('output')
pca_df = pca_model.transform(features)
pca_vector = pca_df.rdd.map(lambda row:row.output)
pcaFeatures = spark.createDataFrame(pca_vector.map(Row),['features'])


Now run K-means with the same parameters as above, but on the ```pcaFeatures``` produced by the PCA reduction you just executed.

Compute the Silhouette score, as well as the number of data points that have been clustered correctly.

In [ ]:
# YOUR CODE HERE
predictions_pca = k_means(pcaFeatures)

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|[-2260.0138862925...|         1|
|[-2368.9937557820...|         1|
|[-2095.6652015478...|         1|
|[-692.69051005705...|         0|
|[-2030.2124927427...|         1|
|[-888.28005357607...|         0|
|[-1921.0822124748...|         1|
|[-1074.7813350047...|         0|
|[-908.57847816188...|         0|
|[-861.57844940756...|         0|
+--------------------+----------+
only showing top 10 rows

Silhouette with squared euclidean distance = 0.8348610363444836


In [ ]:
# YOUR CODE HERE
calculate_TP(predictions_pca,labels,pca=True)


Number of data points that have been clustered correctly after PCA: 486
Percent of data points that have been clustered correctly after PCA: 0.8541300527240774
